# High-Order Example

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/teseoch/fem-intro/master?filepath=fem-intro-high-order.ipynb)
![](QR-ho.png)
Run it with binder!

In [22]:
import numpy as np
import scipy.sparse as spr
from scipy.sparse.linalg import spsolve

import plotly.offline as plotly
import plotly.graph_objs as go
import plotly.figure_factory as ff

#Necessary for the notebook
plotly.init_notebook_mode(connected=True)

As for the linear case the domain is $\Omega = [0, 1]$, which we discretize with $n$ segments (or elements) $s_i$.

Now we also create the high-order nodes.

Note that we append them ad the end.

In [23]:
#domain
omega = np.array([0, 1])

# we now pick the order
order = 3

#number of bases and elements
n_elements = 10
n_bases = n_elements + 1 + n_elements*(order-1)

#Regular nodes, as before
s = np.linspace(omega[0], omega[1], num=n_elements+1)
# s = np.cumsum(np.random.rand(n_elements+1))
# s = (s-s[0])/(s[-1]-s[0])


nodes = s

for e in range(n_elements):
    #For every segment, we create order + 1 new high-order nodes
    tmp = np.linspace(s[e], s[e+1], num=order+1)
    #exclude the first and last since they already exists
    tmp = tmp[1:-1]
    
    #and append at the end of nodes
    nodes = np.append(nodes, tmp)

Plot, in orange the high-order nodes, in blue the linear nodes

In [25]:
fig = go.Figure(data=[
    go.Scatter(x=s, y=np.zeros(s.shape), mode='lines+markers'),
    go.Scatter(x=nodes[n_elements:], y=np.zeros(nodes.shape), mode='markers')
])
plotly.iplot(fig)

# Local bases

As for the linear case we define the **reference element** $\hat s= [0, 1]$, a segment of unit length

On each element we now have `order + 1` (e.g., 2 for linear, 3 for quadratic) **non-zero** local bases.

We define thier "piece" on $\hat s$.

It is important to respect the order of the nodes: the first 2 bases are always for the endpoints, and the other are ordered left to right.

In [27]:
#definition of linear bases, same as before
def hat_phi_1_0(x):
    return 1-x
def hat_phi_1_1(x):
    return x

#definition of quadratic bases
def hat_phi_2_0(x):
    return 2*(x-0.5)*(x-1)
def hat_phi_2_1(x):
    return 2*(x-0)*(x-0.5)
def hat_phi_2_2(x):
    return -4*(x-0.5)**2+1

#definition of cubic bases
def hat_phi_3_0(x):
    return -9/2*(x-1/3)*(x-2/3)*(x-1)
def hat_phi_3_1(x):
    return 9/2*(x-0)*(x-1/3)*(x-2/3)
def hat_phi_3_2(x):
    return 27/2*(x-0)*(x-2/3)*(x-1)
def hat_phi_3_3(x):
    return -27/2*(x-0)*(x-1/3)*(x-1)

#utility function to return the list of functions
def hat_phis(order):
    if order == 1:
        return [hat_phi_1_0, hat_phi_1_1]
    elif order == 2:
        return [hat_phi_2_0, hat_phi_2_1, hat_phi_2_2]
    elif order == 3:
        return [hat_phi_3_0, hat_phi_3_1, hat_phi_3_2, hat_phi_3_3]

We can now plot the `order + 1` local bases bases, same code as before.

Note that the first two correspond to the end-points, and the others are ordered.

In [28]:
x = np.linspace(0, 1)

data = []

tmp = hat_phis(order)

for o in range(order+1):
    data.append(go.Scatter(x=x, y=tmp[o](x), mode='lines', name="$\hat\phi_{}$".format(o)))

fig = go.Figure(data=data)
plotly.iplot(fig)

We use `sympy` to compute the gradients of the local bases

In [29]:
import sympy as sp

xsym = sp.Symbol('x')

def grad_hat_phis(order):
    #For linear we need to get the correct size
    if order == 1:
        return [lambda x : -np.ones(x.shape), lambda x : np.ones(x.shape)]
    
    res = []
    tmp = hat_phis(order)
    
    for fun in tmp:
        res.append(sp.lambdify(xsym, fun(xsym).diff(xsym)))
    return res

Plotting gradients

In [30]:
x = np.linspace(0, 1)

data = []

tmp = grad_hat_phis(order)

for o in range(order+1):
    data.append(go.Scatter(x=x, y=tmp[o](x), mode='lines', name="$\hat\phi_{}$".format(o)))

fig = go.Figure(data=data)
plotly.iplot(fig)

# Basis construction

This code is exacly as before.

The only difficulty is the local to global mapping: 
- the fist 2 nodes are always the same $$g_e^0 = e \qquad\mathrm{and}\qquad g_e^1=g+1$$ 
- the others are 
$$g_e^i = n_{el} + e (\mathrm{order}-1) + i.$$

In [31]:
elements = []
for e in range(n_elements):
    el = {}
    
    el["n_bases"] = order+1
    
    #2 bases
    el["phi"] = hat_phis(order)
    el["grad_phi"] = grad_hat_phis(order)
    
    #local to global mapping
    high_order_nodes = list(range(n_elements + 1 + e*(order-1), n_elements + e*(order-1) + order))
    el["loc_2_glob"] = [e, e+1] + high_order_nodes
    
    #geometric mapping
    el["gmapping"] = lambda x, e=e : s[e] + x*(s[e+1]-s[e])
    el["grad_gmapping"] = lambda x : (s[e+1]-s[e])
    
    elements.append(el)

We define a function to interpolate the $u_i$ using the local to global, geometric mapping, and local bases to interpolate the data. As before

In [32]:
def interpolate(ui):
    u = np.array([])
    x = np.array([])

    xhat = np.linspace(0, 1)


    for e in range(n_elements):
        el = elements[e]
    
        uloc = np.zeros(xhat.shape)

        for i in range(el["n_bases"]):
            glob_node = el["loc_2_glob"][i]
            loc_base = el["phi"][i]
        
            uloc += ui[glob_node] * loc_base(xhat)
    
        u = np.append(u, uloc)
        x = np.append(x, el["gmapping"](xhat))
    
    return x, u

We can generate a random vector $ui$ and use the previous function. This will interpolate all nodes.

In [33]:
ui = np.random.rand(n_bases)

x, u = interpolate(ui)


fig = go.Figure(data=[
    go.Scatter(x=x, y=u, mode='lines'),
    go.Scatter(x=nodes, y=ui, mode='markers'),
])
plotly.iplot(fig)

# Assembly

We are now ready the assemble the global stiffness matrix, which is exacly as before.

In [34]:
import quadpy
scheme = quadpy.line_segment.gauss_patterson(5)

rows = []
cols = []
vals = []

for e in range(n_elements):
    el = elements[e]

    for i in range(el["n_bases"]):
        for j in range(el["n_bases"]):
            val = scheme.integrate(
                lambda x:
                el["grad_phi"][i](x) * el["grad_phi"][j](x) / el["grad_gmapping"](x),
                [0.0, 1.0])
            
            rows.append(el["loc_2_glob"][i])
            cols.append(el["loc_2_glob"][j])
            vals.append(val)

            
rows = np.array(rows)
cols = np.array(cols)
vals = np.array(vals)

L = spr.coo_matrix((vals, (rows, cols)))
L = spr.csr_matrix(L)

We set the row zero and `n_elements` to identity for the boundary conditions

In [35]:
for bc in [0, n_elements]:
    _, nnz = L[bc,:].nonzero()
    for j in nnz:
        if j != bc:
            L[bc, j] = 0.0
    L[bc, bc] = 1.0

We set the righ-hand side to zero, and set the two boundary condition to be 1 and 4

In [36]:
f = np.zeros((n_bases, 1))
f[0] = 1
f[n_elements] = 4

We now solve $Lui=0$ for $ui$

In [37]:
ui = spsolve(L, f)

We now plot, we expect a line, independently from `order`!

In [38]:
x, u = interpolate(ui)


fig = go.Figure(data=[
    go.Scatter(x=x, y=u, mode='lines', name="solution"),
    go.Scatter(x=nodes, y=ui, mode='markers', name="$ui$"),
])
plotly.iplot(fig)

# Mass Matrix

This is exacly as before!

In [39]:
rows = []
cols = []
vals = []

for e in range(n_elements):
    el = elements[e]

    for i in range(el["n_bases"]):
        for j in range(el["n_bases"]):
            val = scheme.integrate(
                lambda x:
                el["phi"][i](x) * el["phi"][j](x) * el["grad_gmapping"](x),
                [0.0, 1.0])
            
            rows.append(el["loc_2_glob"][i])
            cols.append(el["loc_2_glob"][j])
            vals.append(val)

            
rows = np.array(rows)
cols = np.array(cols)
vals = np.array(vals)

M = spr.coo_matrix((vals, (rows, cols)))
M = spr.csr_matrix(M)

Now we set $f=4$ and zero boundary conditions

In [42]:
f = 4*np.ones((n_bases, 1))
f = M*f

f[0] = 0
f[n_elements] = 0

We now solve $Lui=Mf$ for $ui$

In [43]:
ui = spsolve(L, f)

In [44]:
x, u = interpolate(ui)


fig = go.Figure(data=[
    go.Scatter(x=x, y=u, mode='lines', name="solution"),
    go.Scatter(x=nodes, y=ui, mode='markers', name="$ui$"),
])
plotly.iplot(fig)